# Day 6: Guard Gallivant
```
The Historians use their fancy device again, this time to whisk you all away to the North Pole prototype suit manufacturing lab... in the year 1518! It turns out that having direct access to history is very convenient for a group of historians.

You still have to be careful of time paradoxes, and so it will be important to avoid anyone from 1518 while The Historians search for the Chief. Unfortunately, a single guard is patrolling this part of the lab.

Maybe you can work out where the guard will go ahead of time so that The Historians can search safely?

You start by making a map (your puzzle input) of the situation. For example:

....#.....
.........#
..........
..#.......
.......#..
..........
.#..^.....
........#.
#.........
......#...
The map shows the current position of the guard with ^ (to indicate the guard is currently facing up from the perspective of the map). Any obstructions - crates, desks, alchemical reactors, etc. - are shown as #.

Lab guards in 1518 follow a very strict patrol protocol which involves repeatedly following these steps:

If there is something directly in front of you, turn right 90 degrees.
Otherwise, take a step forward.
Following the above protocol, the guard moves up several times until she reaches an obstacle (in this case, a pile of failed suit prototypes):

....#.....
....^....#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
Because there is now an obstacle in front of the guard, she turns right before continuing straight in her new facing direction:

....#.....
........>#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#...
Reaching another obstacle (a spool of several very long polymers), she turns right again and continues downward:

....#.....
.........#
..........
..#.......
.......#..
..........
.#......v.
........#.
#.........
......#...
This process continues for a while, but the guard eventually leaves the mapped area (after walking past a tank of universal solvent):

....#.....
.........#
..........
..#.......
.......#..
..........
.#........
........#.
#.........
......#v..
By predicting the guard's route, you can determine which specific positions in the lab will be in the patrol path. Including the guard's starting position, the positions visited by the guard before leaving the area are marked with an X:

....#.....
....XXXXX#
....X...X.
..#.X...X.
..XXXXX#X.
..X.X.X.X.
.#XXXXXXX.
.XXXXXXX#.
#XXXXXXX..
......#X..
In this example, the guard will visit 41 distinct positions on your map.

Predict the path of the guard. How many distinct positions will the guard visit before leaving the mapped area?
```

In [2]:
import re
import sys
sys.path.append("..")

from common_processing import read_lines, read_as_character_array,\
    read_columns, read_diagonals

test_path = "test.txt"
data_path = "data.txt"

# data = read_lines(test_path)
test_map = read_as_character_array(test_path)
test_map

array([['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '#', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '#', '.', '.', '^', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
       ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.']], dtype='<U1')

In [370]:
# Approach:

""" 
Hope that the guard leaves eventualy so that I don't need to worry about loops
(hope this does not bite me in the @*s later)

Create a copy of the map where we mark with X any tile where the guard has passed.

Update the original map with the current possition of the guard untill it leaves.
"""
import numpy as np 

class Day_6:
    def __init__(self, txt_path: str):

        self.map = read_as_character_array(txt_path)
        self.roadmap = self.map.copy()
        self.direction = self.get_guard_direction()
        self.position = self.get_guard_position()

        pass

    def __str__(self):
        """ 
        Print current state and roadmap
        """
        return str(print("GUARD STATE: \n",self.map,"\n\n ROADMAP: \n", self.roadmap))

    def get_guard_direction(self):
        """ 
        Find one of the possible directions the guard can be facing:
        "^" - UP
        "v" - DOWN
        "<" - LEFT
        ">" - RIGHT
        """
        directions = ["^", "v", "<", ">"]
        for direction in directions:
            if direction in self.map: return direction


    def get_guard_position(self):
        """ 
        Return current possition and direction of the guard in the map.
        We need to know which character the guard is (the direction she's facing)
        """
        return np.argwhere(self.map == self.direction)[0].astype(int)
    
    def lookup_next_tile(self):
        """ 
        Read direction and position, return the very next tile character
        """
        # Check up, down, lef or right
        where_to_go_next = {
            "^": (-1, 0),
            "v": (+1, 0),
            ">": (0, +1),
            "<": (0, -1)
        }

        index_ahead = self.position + where_to_go_next[self.direction]
        return index_ahead
    
    def update_map(self):
        """ 
        Move the guard in the direction she's facing, if it leaves the map, end.

        check the tile directly in fron of the guard

        if it's "#" turn right.
        """
        # Record current position with an X
        self.roadmap[*self.position] = "X"
        
        turn_right = {
            "^": ">",
            "v": "<",
            "<": "^", 
            ">": "v"
        }
        
        self.next_index = self.lookup_next_tile()
        try:
            self.next_tile = self.map[*self.next_index] 
        except:
            print("The guard left")
            return True
        # Base case: swap guard character for "."
        if self.next_tile == ".":
            self.map[*self.position] = "."
            self.map[*self.next_index] = self.direction

        if self.next_tile == "#":
            self.direction = turn_right[self.direction]
            # Also update the map
            self.map[*self.position] = self.direction
            pass

        # Once movement is done:
        self.position = self.get_guard_position()
            

day6 = Day_6(txt_path=test_path)
print(day6)
day6.lookup_next_tile()

GUARD STATE: 
 [['.' '.' '.' '.' '#' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '#']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '#' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '#' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '#' '.' '.' '^' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '#' '.']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '#' '.' '.' '.']] 

 ROADMAP: 
 [['.' '.' '.' '.' '#' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '#']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '#' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '#' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '#' '.' '.' '^' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '#' '.']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '#' '.' '.' '.']]
None


array([5, 4])

In [ ]:
day6.update_map()
print(day6)


The guard left
GUARD STATE: 
 [['.' '.' '.' '.' '#' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '#']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '#' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '#' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '#' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '#' '.']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '#' 'v' '.' '.']] 

 ROADMAP: 
 [['.' '.' '.' '.' '#' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' 'X' 'X' 'X' 'X' 'X' '#']
 ['.' '.' '.' '.' 'X' '.' '.' '.' 'X' '.']
 ['.' '.' '#' '.' 'X' '.' '.' '.' 'X' '.']
 ['.' '.' 'X' 'X' 'X' 'X' 'X' '#' 'X' '.']
 ['.' '.' 'X' '.' 'X' '.' 'X' '.' 'X' '.']
 ['.' '#' 'X' 'X' 'X' 'X' 'X' 'X' 'X' '.']
 ['.' 'X' 'X' 'X' 'X' 'X' 'X' 'X' '#' '.']
 ['#' 'X' 'X' 'X' 'X' 'X' 'X' 'X' '.' '.']
 ['.' '.' '.' '.' '.' '.' '#' '.' '.' '.']]
None


In [371]:
# Loop
i = 0
while i <=1000:
    is_done = day6.update_map()
    if is_done:
        break
    i+=1
print("Iterations: ",i)
print("Final Roadmap: \n",day6.roadmap)
print("Number of tiles, patroled: ", len(np.argwhere(day6.roadmap == "X")) )

The guard left
Iterations:  54
Final Roadmap: 
 [['.' '.' '.' '.' '#' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' 'X' 'X' 'X' 'X' 'X' '#']
 ['.' '.' '.' '.' 'X' '.' '.' '.' 'X' '.']
 ['.' '.' '#' '.' 'X' '.' '.' '.' 'X' '.']
 ['.' '.' 'X' 'X' 'X' 'X' 'X' '#' 'X' '.']
 ['.' '.' 'X' '.' 'X' '.' 'X' '.' 'X' '.']
 ['.' '#' 'X' 'X' 'X' 'X' 'X' 'X' 'X' '.']
 ['.' 'X' 'X' 'X' 'X' 'X' 'X' 'X' '#' '.']
 ['#' 'X' 'X' 'X' 'X' 'X' 'X' 'X' '.' '.']
 ['.' '.' '.' '.' '.' '.' '#' 'X' '.' '.']]
Number of tiles, patroled:  41


In [378]:
# Try witht the big data file (oh god)
day6 = Day_6(txt_path=data_path)
print(day6.map.shape)
# Loop
i = 0
while i <=10000:
    is_done = day6.update_map()
    if is_done:
        break
    i+=1
print("Iterations: ",i)
print("Number of tiles, patroled: ", len(np.argwhere(day6.roadmap == "X")) )


(130, 130)
The guard left
Iterations:  5531
Number of tiles, patroled:  4903


# Part Two
```
While The Historians begin working around the guard's patrol route, you borrow their fancy device and step outside the lab. From the safety of a supply closet, you time travel through the last few months and record the nightly status of the lab's guard post on the walls of the closet.

Returning after what seems like only a few seconds to The Historians, they explain that the guard's patrol area is simply too large for them to safely search the lab without getting caught.

Fortunately, they are pretty sure that adding a single new obstruction won't cause a time paradox. They'd like to place the new obstruction in such a way that the guard will get stuck in a loop, making the rest of the lab safe to search.

To have the lowest chance of creating a time paradox, The Historians would like to know all of the possible positions for such an obstruction. The new obstruction can't be placed at the guard's starting position - the guard is there right now and would notice.

In the above example, there are only 6 different positions where a new obstruction would cause the guard to get stuck in a loop. The diagrams of these six situations use O to mark the new obstruction, | to show a position where the guard moves up/down, - to show a position where the guard moves left/right, and + to show a position where the guard moves both up/down and left/right.

Option one, put a printing press next to the guard's starting position:

....#.....
....+---+#
....|...|.
..#.|...|.
....|..#|.
....|...|.
.#.O^---+.
........#.
#.........
......#...
Option two, put a stack of failed suit prototypes in the bottom right quadrant of the mapped area:


....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
......O.#.
#.........
......#...
Option three, put a crate of chimney-squeeze prototype fabric next to the standing desk in the bottom right quadrant:

....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----+O#.
#+----+...
......#...
Option four, put an alchemical retroencabulator near the bottom left corner:

....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
..|...|.#.
#O+---+...
......#...
Option five, put the alchemical retroencabulator a bit to the right instead:

....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
....|.|.#.
#..O+-+...
......#...
Option six, put a tank of sovereign glue right next to the tank of universal solvent:

....#.....
....+---+#
....|...|.
..#.|...|.
..+-+-+#|.
..|.|.|.|.
.#+-^-+-+.
.+----++#.
#+----++..
......#O..
It doesn't really matter what you choose to use as an obstacle so long as you and The Historians can put it into position without the guard noticing. The important thing is having enough options that you can find one that minimizes time paradoxes, and in this example, there are 6 different positions you could choose.

You need to get the guard stuck in a loop by adding a single new obstruction. How many different positions could you choose for this obstruction?
```

In [ ]:
# Approach:
""" 
Get some logic to check roadmap, 

Roadmap should also record the position and direction

if a position is repeated, check if the direction is also repeated, then we have a loop.
"""

In [450]:

class Day_6:
    def __init__(self, txt_path: str):

        self.map = read_as_character_array(txt_path)
        self.roadmap = self.map.copy()
        self.direction = self.get_guard_direction()
        self.position = self.get_guard_position()
        self.phase_diagram = np.zeros((*self.roadmap.shape, 4))

        pass

    def __str__(self):
        """ 
        Print current state and roadmap
        """
        return str(print("GUARD STATE: \n",self.map,"\n\n ROADMAP: \n", self.roadmap))

    def get_guard_direction(self):
        """ 
        Find one of the possible directions the guard can be facing:
        "^" - UP
        "v" - DOWN
        "<" - LEFT
        ">" - RIGHT
        """
        directions = ["^", "v", "<", ">"]
        for direction in directions:
            if direction in self.map: return direction


    def get_guard_position(self):
        """ 
        Return current possition and direction of the guard in the map.
        We need to know which character the guard is (the direction she's facing)
        """
        return np.argwhere(self.map == self.direction)[0].astype(int)
    
    def lookup_next_tile(self):
        """ 
        Read direction and position, return the very next tile character
        """
        # Check up, down, lef or right
        where_to_go_next = {
            "^": (-1, 0),
            "v": (+1, 0),
            ">": (0, +1),
            "<": (0, -1)
        }

        index_ahead = self.position + where_to_go_next[self.direction]
        return index_ahead
    
    def update_pase_diagram(self):
        """ 
        Phase diagram is n x n x 4, one extra dim to encode
        direction.

        Phase diagram starts empty, 
        whenenver map updates, add a 1 to the corresponding dimention based
        on the guard's direction
        """
        direction2dimention = {
            "^": 0,
            "v": 1,
            ">": 2,
            "<": 3
        }
        # Record phase diagram, if it has been already written we have a loop
        if self.phase_diagram[*self.position, direction2dimention[self.direction]] == 1:
            return True
        else:
            self.phase_diagram[*self.position, direction2dimention[self.direction]] = 1
            return False
    
    def update_map(self):
        """ 
        Move the guard in the direction she's facing, if it leaves the map, end.

        check the tile directly in fron of the guard

        if it's "#" turn right.
        """
        # Record current position with an X
        self.roadmap[*self.position] = "X"
        looping = self.update_pase_diagram()

        if looping:
            print("The guard is looping")
            return "Looping"
        
        turn_right = {
            "^": ">",
            "v": "<",
            "<": "^", 
            ">": "v"
        }
        
        self.next_index = self.lookup_next_tile()
        # Check if the new index is inside the map
        i, j = self.next_index
        n, m = self.map.shape
        if 0 <= i < n and 0 <= j < m:
            self.next_tile = self.map[*self.next_index] 
        else:
            print("The guard left")
            return True
        # Base case: swap guard character for "."
        if self.next_tile == ".":
            self.map[*self.position] = "."
            self.map[*self.next_index] = self.direction

        if self.next_tile == "#" or self.next_tile == "O":
            self.direction = turn_right[self.direction]
            # Also update the map
            self.map[*self.position] = self.direction
            pass

        # Once movement is done:
        self.position = self.get_guard_position()
            

day6 = Day_6(txt_path=test_path)

In [451]:
# Add an obstacle on a tile
day6 = Day_6(txt_path=test_path)
print(day6)

print("--- Add a 'O'bstacle on some place --- ")
some_place = [6, 8]
day6.map[*some_place] = "O"
day6.roadmap[*some_place] = "O"

print(day6)


GUARD STATE: 
 [['.' '.' '.' '.' '#' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '#']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '#' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '#' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '#' '.' '.' '^' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '#' '.']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '#' '.' '.' '.']] 

 ROADMAP: 
 [['.' '.' '.' '.' '#' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '#']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '#' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '#' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '#' '.' '.' '^' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '#' '.']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '#' '.' '.' '.']]
None
--- Add a 'O'bstacle on some place --- 
GUARD STATE: 
 [['.' '.' '.' '.' '#' '.' '.' '.' '.' '.']
 ['.' 

In [452]:
print(day6.map.shape)
# Loop
i = 0
loops_found = 0
while i <=10000:
    is_done = day6.update_map()
    if is_done =="Looping":
        loops_found +=1
        break
    
    if is_done:
        break
    i+=1
print("Iterations: ",i)
print("Final Roadmap: \n",day6)
print("Number of tiles, patroled: ", len(np.argwhere(day6.roadmap == "X")) )
print("Number of loops: ", loops_found)

(10, 10)
The guard left
Iterations:  24
Final Roadmap: 
 GUARD STATE: 
 [['.' '.' '.' '.' '#' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '#']
 ['.' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '#' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '.' '#' '.' '.']
 ['<' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '#' '.' '.' '.' '.' '.' '.' 'O' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '#' '.']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '#' '.' '.' '.']] 

 ROADMAP: 
 [['.' '.' '.' '.' '#' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' 'X' 'X' 'X' 'X' 'X' '#']
 ['.' '.' '.' '.' 'X' '.' '.' '.' 'X' '.']
 ['.' '.' '#' '.' 'X' '.' '.' '.' 'X' '.']
 ['.' '.' '.' '.' 'X' '.' '.' '#' 'X' '.']
 ['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' '.']
 ['.' '#' '.' '.' 'X' '.' '.' '.' 'O' '.']
 ['.' '.' '.' '.' '.' '.' '.' '.' '#' '.']
 ['#' '.' '.' '.' '.' '.' '.' '.' '.' '.']
 ['.' '.' '.' '.' '.' '.' '#' '.' '.' '.']]
None
Number of tiles, patroled:  21
Number of loops:

In [ ]:
# Run on every single possible obstacle:
""" Could this be pallalelized? """
day6 = Day_6(txt_path=test_path)
all_indeces = np.ndindex(day6.map.shape)
starting_position = np.argwhere(day6.map == "^")[0]

loops_found = 0
for j in all_indeces:
    if j == (starting_position[0].astype(int), starting_position[1].astype(int)):
        continue

    day6 = Day_6(txt_path=test_path)
    # Add an obstacle at the index
    day6.map[*j] = "O"
    day6.roadmap[*j] = "O"

    i = 0
    while i <=10000:
        is_done = day6.update_map()
        if is_done =="Looping":
            loops_found +=1
            print("Obstacle at: ", j)
            print(day6)
            break
        
        if is_done:
            break
        i+=1
        if i == 10000:
            print("Max Iterations: ",i) 

print("Number of loops: ", loops_found)

The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard is looping
Obstacle at:  (6, 3)
GUARD STATE: 

## I DONT NEED TO CHECK EVERY SPOT!
Only the ones that the original path crosses

In [ ]:
# Compute the original path
day6 = Day_6(txt_path=data_path)
print(day6.map.shape)
# Loop
i = 0
while i <=10000:
    is_done = day6.update_map()
    if is_done:
        break
    i+=1
print("Iterations: ",i)
print("Number of tiles, patroled: ", len(np.argwhere(day6.roadmap == "X")) )

all_indeces = np.where(day6.roadmap == "X")
all_indeces = [(i, j) for i,j in zip(*all_indeces)]

In [472]:
day6 = Day_6(txt_path=data_path)
# all_indeces = np.ndindex(day6.map.shape)
starting_position = np.argwhere(day6.map == "^")[0]

loops_found = 0
for j in all_indeces:
    if j == (starting_position[0].astype(int), starting_position[1].astype(int)):
        continue

    day6 = Day_6(txt_path=data_path)
    # Add an obstacle at the index
    day6.map[*j] = "O"
    day6.roadmap[*j] = "O"

    i = 0
    while i <=10000:
        is_done = day6.update_map()
        if is_done =="Looping":
            loops_found +=1
            print("Obstacle at: ", j)
            # print(day6)
            break
        
        if is_done:
            break
        i+=1
        if i == 10000:
            print("Max Iterations: ",i) 

print("Number of loops: ", loops_found)


The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard is looping
Obstacle at:  (np.int64(9), np.int64(23))
The guard left
The guard is looping
Obstacle at:  (np.int64(9), np.int64(25))
The guard left
The guard left
The guard is looping
Obstacle at:  (np.int64(9), np.int64(28))
The guard left
The guard left
The guard is looping
Obstacle at:  (np.int64(9), np.int64(31))
The guard left
The guard left
The guard left
The guard left
The guard left
The guard is looping
Obstacle at:  (np.int64(9), np.int64(37))
The guard left
The guard is looping
Obstacle at:  (np.int64(9), np.int64(39))
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard left
The guard is looping
Obstacle at:  (np.int64(9), np.int64(47))
The guard left
The guard is looping
Obstacle at:  (np.int64(9), np.int64(49))
The guard left
The guard is looping
Obstacle at:  (np.int64(9), np.int64(51))
The guard is looping
Obstacl

This brute force could be improved:

An obstacle loops if:
- the original path encounters it head on
- The new obstacle forms an "ofset rectangle" with existing obstacles.
```
....O.....
.........O
..........
..#.......
.......#..
..........
.#.N^.....
........O.
#.........
......#...
```

<!-- ## This is hard to debug, maybe add `O` as the obstacle and baheve the same as `#` -->